In [1]:
#ThermostatSC2

In [2]:
import datetime
import time
import random
import logging
import uuid

from utils import *

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
THERMOSTAT_API_URL = "http://0.0.0.0:8001/thermostat"
PREFIXES={
        "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        "saref": "https://w3id.org/saref#",
    }

GRAPH_PATTERN = """?meas rdf:type saref:Measurement .
                     ?meas saref:hasValue ?temp .
                     ?meas saref:isMeasuredIn saref:TemperatureUnit .
                     ?meas saref:hasTimestamp ?timestamp .
                     ?meas saref:isMeasurementOf ?room_id .
                     ?meas saref:relatesToProperty saref:Temperature .
                     ?meas saref:measurementMadeBy ?device_id ."""

In [4]:
def get_temp_now():
    return requests.get(THERMOSTAT_API_URL + "/actual_temperature").json()

In [5]:

def start_sensor_kb(kb_id, kb_name, kb_description, ke_endpoint):
    register_knowledge_base(kb_id, kb_name, kb_description, ke_endpoint)
    post_ki_id = register_post_knowledge_interaction(
        GRAPH_PATTERN,
        None,
        "post-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    measurement_counter = 0
    # two threads?
    while True:
        measurement_counter += 1
        value = get_temp_now()

        post(
            [
                {
                    "meas": f"<{THERMOSTAT_API_URL}/measurements/{str(uuid.uuid4())}>",
                    "temp": f"{get_temp_now()}",
                    "timestamp": f'"{get_timestamp_now()}"',  # ISO 8601 format
                    "room_id": f'"{THERMOSTAT_API_URL}/rooms/1"',
                    "device_id": f'"{THERMOSTAT_API_URL}/devices/1"',
                }
            ],
            post_ki_id,
            kb_id,
            ke_endpoint,
        )
        logger.info(f"published measurement of {value} units at {get_timestamp_now()}")

        time.sleep(5)


In [6]:
def handle_answer_measurements(query_bindings):
    global measurements
    print(f'Answer query bindings: {query_bindings}')
    #change desired temp
    requests.put(
        THERMOSTAT_API_URL
        + "/desired_temperature"
        + f'?desired_temperature={query_bindings["desired_temp"]}'
    )
    return match_bindings(query_bindings, measurements)

In [7]:
import threading

def start_sensor_kb(kb_id, kb_name, kb_description, ke_endpoint):
    register_knowledge_base(kb_id, kb_name, kb_description, ke_endpoint)
    post_ki_id = register_post_knowledge_interaction(
        GRAPH_PATTERN,
        None,
        "post-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )
    
    answer_measurements_ki = register_answer_knowledge_interaction(
        GRAPH_PATTERN,
        "answer-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    measurement_counter = 0

    # Start the start_handle_loop function in a separate thread
    handle_loop_thread = threading.Thread(target=start_handle_loop, args=(
        {
            answer_measurements_ki: handle_answer_measurements,
        },
        kb_id,
        ke_endpoint,
    ))
    handle_loop_thread.start()

    # Main loop for posting measurements
    while True:
        measurement_counter += 1
        value = get_temp_now()

        post(
            [
                {
                    "meas": f"<{THERMOSTAT_API_URL}/measurements/{str(uuid.uuid4())}>",
                    "temp": f"{get_temp_now()}",
                    "timestamp": f'"{get_timestamp_now()}"',  # ISO 8601 format
                    "room_id": f'"{THERMOSTAT_API_URL}/rooms/1"',
                    "device_id": f'"{THERMOSTAT_API_URL}/devices/1"',
                }
            ],
            post_ki_id,
            kb_id,
            ke_endpoint,
        )
        logger.info(f"published measurement of {value} units at {get_timestamp_now()}")

        time.sleep(5)


In [8]:
start_sensor_kb(
    "http://example.org/sensor",
    "Thermostat1_temp",
    "A thermostat",
    "http://knowledge_engine:8280/rest/",
)

INFO:utils:registered Thermostat1_temp
INFO:utils:received issued knowledge interaction id: http://example.org/sensor/interaction/post-measurements
INFO:utils:received issued knowledge interaction id: http://example.org/sensor/interaction/answer-measurements
INFO:__main__:published measurement of 21 units at 2023-06-19T01:08:20+00:00
INFO:__main__:published measurement of 19 units at 2023-06-19T01:08:26+00:00
INFO:__main__:published measurement of 20 units at 2023-06-19T01:08:32+00:00
INFO:__main__:published measurement of 18 units at 2023-06-19T01:08:38+00:00
INFO:__main__:published measurement of 20 units at 2023-06-19T01:08:43+00:00
INFO:__main__:published measurement of 17 units at 2023-06-19T01:08:49+00:00
INFO:__main__:published measurement of 23 units at 2023-06-19T01:08:55+00:00
INFO:__main__:published measurement of 19 units at 2023-06-19T01:09:01+00:00
INFO:__main__:published measurement of 21 units at 2023-06-19T01:09:07+00:00
INFO:__main__:published measurement of 22 units 

KeyboardInterrupt: 